<a href="https://colab.research.google.com/github/MyeongGuJo/Transformer/blob/main/Transformer_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import pandas as pd

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
tokenizer('I am a student of Hanyang University', add_special_tokens=True)
encoded = tokenizer('I am an undergraduated', return_tensors='pt')
print(encoded)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

{'input_ids': tensor([[ 101, 1045, 2572, 2019, 8324, 2094,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}


In [4]:
'''
d = 4
embedding_layer = nn.Embedding(num_embeddings=len(tokenizer.vocab),
                               embedding_dim=4,
                               padding_idx=4)

input_seq = embedding_layer(encoded['input_ids'])
print(input_seq, input_seq.shape)
'''

"\nd = 4\nembedding_layer = nn.Embedding(num_embeddings=len(tokenizer.vocab),\n                               embedding_dim=4,\n                               padding_idx=4)\n\ninput_seq = embedding_layer(encoded['input_ids'])\nprint(input_seq, input_seq.shape)\n"

In [5]:
'''
def calculate_attention(Q, K, V, Mask = None):
  bs = Q.shape[0]

#  print(f'Q.shape: {Q.shape}')
#  print(f'K.T.shape: {K.transpose(-1, -2).shape}')
  energy = Q.matmul(K.transpose(-1, -2))
#  print(f'energy: {energy.shape}')

  # scaling
  d_k = K.size(-1)**(1/2)
#  print(f'd_k: {d_k}')
  energy = energy / d_k

  if Mask is not None:
    energy = energy.masked_fill(Mask == 0, -1e+9)
#    print(f'energy\n{energy}\n{energy.shape}')

  attention = F.softmax(energy, -1)
#  print(f'attention\n{attention}')
  attention = attention.matmul(V)

  return attention
'''

"\ndef calculate_attention(Q, K, V, Mask = None):\n  bs = Q.shape[0]\n\n#  print(f'Q.shape: {Q.shape}')\n#  print(f'K.T.shape: {K.transpose(-1, -2).shape}')\n  energy = Q.matmul(K.transpose(-1, -2))\n#  print(f'energy: {energy.shape}')\n\n  # scaling\n  d_k = K.size(-1)**(1/2)\n#  print(f'd_k: {d_k}')\n  energy = energy / d_k\n\n  if Mask is not None:\n    energy = energy.masked_fill(Mask == 0, -1e+9)\n#    print(f'energy\n{energy}\n{energy.shape}')\n\n  attention = F.softmax(energy, -1)\n#  print(f'attention\n{attention}')\n  attention = attention.matmul(V)\n\n  return attention\n"

In [6]:
'''
# d == 4
q_proj = nn.Linear(d, d, bias=False)
k_proj = nn.Linear(d, d, bias=False)
v_proj = nn.Linear(d, d, bias=False)
print(input_seq.shape)
query = q_proj(input_seq)
key = k_proj(input_seq)
value = v_proj(input_seq)
print(query.shape)

#mask
m = torch.tril(torch.ones((value.size(1), value.size(1)), dtype=int))
print(m)
print(calculate_attention(query, key, value, Mask=m))
'''

'\n# d == 4\nq_proj = nn.Linear(d, d, bias=False)\nk_proj = nn.Linear(d, d, bias=False)\nv_proj = nn.Linear(d, d, bias=False)\nprint(input_seq.shape)\nquery = q_proj(input_seq)\nkey = k_proj(input_seq)\nvalue = v_proj(input_seq)\nprint(query.shape)\n\n#mask\nm = torch.tril(torch.ones((value.size(1), value.size(1)), dtype=int))\nprint(m)\nprint(calculate_attention(query, key, value, Mask=m))\n'

In [7]:
'''
def feed_forward(x, d):
  inner_proj = nn.Linear(d, d*4, bias=False)
  outer_proj = nn.Linear(d*4, d, bias=False)
  relu = F.relu()

  inner = inner_proj(x)
  output = relu(inner)

  output = outer_proj(output)

  return output
'''

'\ndef feed_forward(x, d):\n  inner_proj = nn.Linear(d, d*4, bias=False)\n  outer_proj = nn.Linear(d*4, d, bias=False)\n  relu = F.relu()\n  \n  inner = inner_proj(x)\n  output = relu(inner)\n\n  output = outer_proj(output)\n\n  return output\n'

In [8]:
'''
output = calculate_attention(query, key, value, Mask=m)
residual = output

output = residual + output

# add
print(output)

# LayerNormalization
layer_norm = nn.LayerNorm(d)
output = layer_norm(output)

print(output)
'''

'\noutput = calculate_attention(query, key, value, Mask=m)\nresidual = output\n\noutput = residual + output\n\n# add\nprint(output)\n\n# LayerNormalization\nlayer_norm = nn.LayerNorm(d)\noutput = layer_norm(output)\n\nprint(output)\n'

In [9]:
class Attention(nn.Module):
  def __init__(self, dim):
    super().__init__()
    self.q_proj = nn.Linear(dim)
    self.k_proj = nn.Linear(dim)
    self.v_proj = nn.Linear(dim)

  def forward(self, Q, K, V, Mask = None):
    query = self.q_proj(Q)
    key = self.k_proj(K)
    value = self.v_proj(V)

    bs = query.shape[0]

    energy = query.matmul(key.transpose(-1, -2))

    # scaling
    d_k = key.size(-1)**(1/2)
    energy = energy / d_k

    if Mask is not None:
      energy = energy.masked_fill(Mask == 0, -1e9)

    output = F.softmax(energy, -1)
    output = output.matmul(value)

    return output

In [10]:
class FeedForward(nn.Module):
  def __init__(self, d):
    super().__init__()
    self.inner_proj = nn.Linear(d, d*4, bias=False)
    self.outer_proj = nn.Linear(d*4, d, bias=False)
    self.relu = F.relu()

  def forward(self, x):
    inner = self.inner_proj(x)
    output = self.relu(inner)

    output = self.outer_proj(output)

    return output

In [11]:
class Encoder(nn.Module):
  def __init__(self, vocab_size, dim):
    super().__init__()
    self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=dim, padding_idx=0)
    self.pos_embedding = nn.Embedding(100, dim) # max_length = 100

    self.layer_norm = nn.LayerNorm(dim)

    self.self_attention = Attention(dim)

    self.feed_forward = FeedForward(dim)

  def forward(self, input):
    input_seq = self.embedding(input) + self.pos_embedding(input)

    # self attention
    residual = output
    output = self.self_attention(input_seq, input_seq, input_seq, Mask=m)

    # Add & Norm
    output = self.layer_norm(output)
    output = residual + output

    # Feed Forward
    residual = output
    output = self.feed_forward(output)

    # Add & Norm
    output = residual + output
    output = self.layer_norm(output)

    return output

In [12]:
class Decoder(nn.Module):
  def __init__(self, vocab_size, dim):
    super().__init__()
    self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=dim, padding_idx=0)
    self.pos_embedding = nn.Embedding(100, dim) # max_length = 100

    self.layer_norm = nn.LayerNorm(dim)

    self.self_attention = Attention(dim)
    self.cross_attention = Attention(dim)

    self.feed_forward = FeedForward(dim)

    self.out_proj = nn.Linear(dim)

  def forward(self, input, enc_output):
    input_seq = self.embedding(input) + self.pos_embedding(input)

    # mask
    m = torch.tril(torch.ones((value.size(1), value.size(1)), dtype=int))

    # self attention
    residual = output
    output = self.self_attention(input_seq, input_seq, input_seq, Mask=m)

    # Add & Norm
    output = self.layer_norm(output)
    output = residual + output

    # cross attention
    residual = output
    output = self.cross_attention(output, enc_output, enc_output)

    # Add & Norm
    output = residual + output
    output = self.layer_norm(output)

    # Feed Forward
    residual = output
    output = self.feed_forward(output)

    # Add & Norm
    output = residual + output
    output = self.layer_norm(output)

    logits = output

    # Linear & Softmax
    output = self.out_proj(output)
    output = F.softmax(output, -1)

    output_token = torch.argmax(output, dim=-1)

    return logits, output_token

In [13]:
class Transformer(nn.Module):
  def __init__(self, vocab_size = 3200, dim = 256):
    super().__init__()
    self.encoder = Encoder(vocab_size, dim)
    self.decoder = Decoder(vocab_size, dim)

  def forward(self, enc_src, dec_src):
    # enc_src: (bs, seq_len)
    # dec_src: (bs, seq_len)

    enc_output = self.encoder(enc_src)
    logits, output = self.decoder(dec_src, enc_output)

    return logits, output